347: Python function to count the number of squares in a rectangle.

In [1]:
%%writefile bugged_count_squares.py

def count_Squares(m,n):
  if (n < m):
    temp = m
    m = n
    n = temp
  return n * (n + 1) * (3 * m - n) // 6   #bug: (3m-n) instead of (3m-n+1)

Writing bugged_count_squares.py


LLM TESTING

In [2]:
%%writefile llm_test_count_squares.py

import pytest
from bugged_count_squares import count_Squares


def test_square_1x1():
    # Only one square
    assert count_Squares(1, 1) == 1


def test_square_2x2():
    # 1x1 squares: 4, 2x2 squares: 1 → total = 5
    assert count_Squares(2, 2) == 5


def test_square_3x3():
    # 1x1: 9, 2x2: 4, 3x3: 1 → total = 14
    assert count_Squares(3, 3) == 14


def test_rectangle_2x3():
    # Formula sum of (m−k+1)(n−k+1) for k up to min(m,n)
    # k=1: 2*3 = 6
    # k=2: 1*2 = 2
    # total = 8
    assert count_Squares(2, 3) == 8


def test_rectangle_3x4():
    # k=1: 3*4 = 12
    # k=2: 2*3 = 6
    # k=3: 1*2 = 2
    # total = 20
    assert count_Squares(3, 4) == 20


def test_symmetry_property():
    # count_Squares(m, n) == count_Squares(n, m)
    assert count_Squares(4, 6) == count_Squares(6, 4)


def test_one_dimension_large():
    # 1 x N grid always has N squares
    assert count_Squares(1, 10) == 10
    assert count_Squares(10, 1) == 10


def test_zero_dimension():
    # Expected to be 0 squares if any side is zero
    assert count_Squares(0, 5) == 0
    assert count_Squares(5, 0) == 0


def test_large_square():
    # 4x4 → 1x1:16, 2x2:9, 3x3:4, 4x4:1 → total = 30
    assert count_Squares(4, 4) == 30

Writing llm_test_count_squares.py


In [3]:
!pytest -v llm_test_count_squares.py

============================= test session starts =============================
platform win32 -- Python 3.13.2, pytest-8.4.2, pluggy-1.6.0 -- C:\Python313\python.exe
cachedir: .pytest_cache
hypothesis profile 'default'
rootdir: d:\IIIT HYD\SSD\Final project\HumanVsLLM\347
plugins: anyio-4.9.0, hypothesis-6.147.0
collecting ... collected 9 items

llm_test_count_squares.py::test_square_1x1 FAILED                        [ 11%]
llm_test_count_squares.py::test_square_2x2 FAILED                        [ 22%]
llm_test_count_squares.py::test_square_3x3 FAILED                        [ 33%]
llm_test_count_squares.py::test_rectangle_2x3 FAILED                     [ 44%]
llm_test_count_squares.py::test_rectangle_3x4 FAILED                     [ 55%]
llm_test_count_squares.py::test_symmetry_property PASSED                 [ 66%]
llm_test_count_squares.py::test_one_dimension_large FAILED               [ 77%]
llm_test_count_squares.py::test_zero_dimension FAILED                    [ 88%]
llm_test_co

HUMAN TESTING

In [ ]:
%%writefile human_test_count_squares.py
from hypothesis import given, strategies as st, settings
import pytest
from bugged_count_squares import count_Squares


@given(st.integers(min_value=0, max_value=500), st.integers(min_value=0, max_value=500))
def test_commutative(m, n):
    assert count_Squares(m, n) == count_Squares(n, m)

@given(st.integers(min_value=1, max_value=500), st.integers(min_value=1, max_value=500))
def test_non_negative_integer(m, n):
    result = count_Squares(m, n)
    assert isinstance(result, int)
    assert result >= 0

@given(st.integers(min_value=0, max_value=120), st.integers(min_value=0, max_value=120))
@settings(max_examples=300)
def test_canonical_sum_identity_inline(m, n):
    total = 0
    limit = m if m < n else n
    k = 1
    while k <= limit:
        total += (m - k + 1) * (n - k + 1)
        k += 1
    assert count_Squares(m, n) == total

@given(
    st.integers(min_value=0, max_value=200),
    st.integers(min_value=0, max_value=200),
    st.integers(min_value=0, max_value=500) 
)
def test_monotonicity(m, k, n):
    m2 = m + k
    assert count_Squares(m2, n) >= count_Squares(m, n)


@given(st.integers(min_value=0, max_value=400))
def test_square_grid(k):
    expected = k * (k + 1) * (2 * k + 1) // 6
    assert count_Squares(k, k) == expected

Overwriting human_test_count_squares.py


In [8]:
!pytest -v human_test_count_squares.py

============================= test session starts =============================
platform win32 -- Python 3.13.2, pytest-8.4.2, pluggy-1.6.0 -- C:\Python313\python.exe
cachedir: .pytest_cache
hypothesis profile 'default'
rootdir: d:\IIIT HYD\SSD\Final project\HumanVsLLM\347
plugins: anyio-4.9.0, hypothesis-6.147.0
collecting ... collected 5 items

human_test_count_squares.py::test_symmetry_commutativity PASSED          [ 20%]
human_test_count_squares.py::test_non_negative_and_integer FAILED        [ 40%]
human_test_count_squares.py::test_canonical_sum_identity_inline FAILED   [ 60%]
human_test_count_squares.py::test_monotonicity_in_m FAILED               [ 80%]
human_test_count_squares.py::test_square_grid_closed_form FAILED         [100%]

================================== FAILURES ===================================
________________________ test_non_negative_and_integer ________________________

    @given(st.integers(min_value=1, max_value=500), st.integers(min_value=1, max_value=50